<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# Lab 2 - Testing the database

As usual we have to tell Jupyter to allow SQL:

In [2]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [3]:
%sql sqlite:///movies.db

'Connected: @movies.db'

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [4]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.db
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [8]:
%%sql
select title from films;

 * sqlite:///movies.db
Done.


title
Star Wars
Raiders of the Lost Ark
Iron Sky


+ Show the performance dates for one of the movies.

In [12]:
%%sql
select date, title, t_name
from performances 
join films
using (imdb_key)
where title like 'Star Wars'

 * sqlite:///movies.db
Done.


date,title,t_name
2019-02-15,Star Wars,Filmstaden
2019-02-15,Star Wars,Filmstaden


+ Show all data concerning performances at a given theatere
  on a given date.

In [13]:
%%sql
select * 
from performances 
where t_name like 'Kino'AND date like '2019-02-15'

 * sqlite:///movies.db
Done.


p_id,date,time,imdb_key,t_name
2,2019-02-15,19:20,tt1034314,Kino
fa4ce0e6258148fd39d2a7b14c333d2f,2019-02-15,21:55,tt1034314,Kino


+ List all customers

In [15]:
%%sql
select * from customers

 * sqlite:///movies.db
Done.


user_name,full_name,password
dat15asy,Axel Syrén,59519f9837117c5a1fa2be0192555a69


+ List all tickets

In [22]:
%%sql
select * from tickets

 * sqlite:///movies.db
Done.


t_id,p_id,user_name
0df8713520d95bbf058a7c26f3adbab9,1,dat15asy
813891c0a59a8ae7805d1cdae6bd475e,2,dat15asy
c4e8b6ad011450945322ba555a70d522,3,dat14sry


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [21]:
%%sql
INSERT 
INTO customers (user_name, full_name)
VALUES ('dat14sry', 'Simon Rydebrink');
INSERT 
INTO tickets (p_id, user_name)
VALUES (3, 'dat14sry')

 * sqlite:///movies.db
1 rows affected.
1 rows affected.


[]

In
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html)
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


In SQLite3 (as of January 2019), we can't do that, instead
  we can the following idea: each row in a SQLite3 table has
  a `rowid` attribute, it is a unique integer which
  essentially tells in which order the rows were inserted,
  and it's not displayed in queries, unless we ask for it.
  SQLite3 also have a function, `last_insert_rowid()`, which
  returns the `rowid` of the last inserted row of a table,
  so we can see the `s_id` of the most recently inserted
  student with the following query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [23]:
%%sql
select t_id
from tickets
where rowid = last_insert_rowid();

 * sqlite:///movies.db
Done.


t_id
c4e8b6ad011450945322ba555a70d522


+ Try to insert two movie theaters with the same name (this
  should fail).

In [25]:
%%sql
INSERT 
INTO theatres (t_name, capacity)
VALUES ('MountainKing', 666),
        ('MountainKing', 420);


 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theatres.t_name [SQL: "INSERT \nINTO theatres (t_name, capacity)\nVALUES ('MountainKing', 666),\n        ('MountainKing', 420);"] (Background on this error at: http://sqlalche.me/e/gkpj)

+ Try to insert a performance where the theater doesn’t
  exist in the database (this should fail).

In [27]:
%%sql
INSERT 
INTO performances (date, time, imdb_key, t_name)
VALUES ('2019-02-15', '16:45', 'tt1034314', 'MovieHouse')

 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "INSERT \nINTO performances (date, time, imdb_key, t_name)\nVALUES ('2019-02-15', '16:45', 'tt1034314', 'MovieHouse')"] (Background on this error at: http://sqlalche.me/e/gkpj)

+ Create a ticket where either the user or the performance
  doesn’t exist.

In [28]:
%%sql
INSERT 
INTO tickets (p_id, user_name)
VALUES (3, 'dat15vcl');


 * sqlite:///movies.db


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "INSERT \nINTO tickets (p_id, user_name)\nVALUES (3, 'dat15vcl');"] (Background on this error at: http://sqlalche.me/e/gkpj)